#### COMP 215 - Project 02

# Extending the Forest Fire Model

#### Name: Ben Blair
#### Date: 13/04/2023

## Background:

In [1990](https://doi.org/10.1016/0375-9601(90)90451-S) Bak, Chen and Tang proposed a cellular automaton that is an abstract model of a forest fire. Each cell is in one of three states: empty, occupied by forest, or on fire.

3 ecological processes are modelled: forest regeneration, fire ignition, and fire spread

  * empty cells "regenerate" forest at a fixed rate, $p$
  * forest fires ignite with a regular but small frequency, $f$
  * forested cells catch fire when their neighbours are burning, and burn out in one time step.

## Project Description:

## Instructions:

## Import modules



The first code block imports all of the modules we need.

In [1]:
%matplotlib inline

import time

import matplotlib
import matplotlib.pyplot as plt
from matplotlib import animation

import numpy as np
from scipy.signal import correlate2d

# Configure matplotlib's animation library to work in the browser.
matplotlib.rc('animation', html='jshtml')

## Helpers

Some useful, reusable code snippets to help visualize the model.

In [2]:
DEFAULT_IMSHOW_OPTIONS = dict(
    cmap='Blues',
    interpolation='none',
    origin='upper',
)

def plot_2D_array(array, axes=None, title='', **options):
    """ Plot the 2D array as an image on the given axes  1's will be dark blue, 0's will be light blue. """
    axes = axes or plt.gca()  # If not axes are provided, draw on current axes
    axes.set_title(title)
    axes.set_xticks([], [])  # remove axes tick marks
    axes.set_yticks([], [])
    options = {**DEFAULT_IMSHOW_OPTIONS, **options}
    axes.imshow(array, **options)

In [3]:
class Animation2D:
    """
      Animates any 2D model with a step() method and a draw() method, using matplotlib
      model.step() should take no parameters - just step the model forward one step.
      model.draw() should take 2 parameters, the matpltolib axes to draw on and an integer step number

      See https://www.allendowney.com/blog/2019/07/25/matplotlib-animation-in-jupyter/
          for a discussion of the pros and cons of various animation techniques in jupyter notebooks
    """

    def __init__(self, model, frames=50, steps_per_frame=1, figsize=(8, 8)):
        """
        :param model: the simulation object to animate, with step() and draw(axes, step) methods
        :param frames: number of animation frames to generate
        """
        self.model = model
        self.frames = frames
        self.steps_per_frame = steps_per_frame
        self.fig, self.ax = plt.subplots(figsize=figsize)

    def animation_step(self, step):
        """ Step the model forward and draw the plot """
        if step > 0:
            for _ in range(self.steps_per_frame):
                self.model.step()
        self.model.draw(self.ax, step=step * self.steps_per_frame)

    def show(self):
        """ return the matplotlib animation object, ready for display """
        anim = animation.FuncAnimation(self.fig, self.animation_step, frames=self.frames)
        plt.close()  # this ensures the last frame is not shown as a separate plot
        return anim

    def animate(self, interval=None):
        """ Animate the model simulation directly in the notebook display block """
        from IPython.display import clear_output
        try:
            for i in range(self.frames):
                clear_output(wait=True)  # clear the IPython display
                self.ax.clear()  # clear old image from the axes (fixes a performance issue)
                plt.figure(self.fig)  # add the figure back to pyplot ** sigh **
                self.animation_step(i)
                plt.show()  # show the current animation frame (pyplot then closes and throws away figure ** sigh **)
                if interval:
                    time.sleep(interval)
        except KeyboardInterrupt:
            pass

## Build the ForestFire model

1. Define the kernel (correlation matrix)

    * What do we need to know about a cell's neighbourhood?
    * How can we encode that using a dot product with a 3 x 3 correlation matrix?

2. The step function will be more complex because it must implement the 4 rules:

    *  An empty cell becomes occupied with probability $p$.
    *  A cell with a tree burns if any of its neighbors is on fire.
    *  A cell with a tree spontaneously burns, with probability $f$, even if none of its neighbors is on fire.
    *  A cell with a burning tree becomes an empty cell in the next time step.

Typical values for the parameters are $p=0.01$ and $f=0.001$

In [4]:
class ForestFire:
    """ 2D Cellular Automaton that simulates a fire-dominated landscape. """

    # Define names for the 3 possible cell states
    EMPTY = 0
    OCCUPIED = 1
    FIRE = 5

    # Define a colour map that maps each cell state to an intuitive colour.
    cmap = [(0.5, 0.3, 0), (0.3, 0.6, 0), (0, 0, 0), (0, 0, 0), (0, 0, 0), (0.9, 0, 0)]
    cmap[EMPTY] = (0.5, 0.3, 0)  # brown
    cmap[OCCUPIED] = (0.3, 0.6, 0)  # green
    cmap[FIRE] = (0.9, 0, 0)  # red
    forest_colour_map = matplotlib.colors.ListedColormap(cmap)

    # Define a correlation kernel --> same as the percolation model
    kernel = np.array([[0, 1, 0],
                       [1, 0, 1],
                       [0, 1, 0]])
    

    def __init__(self, n, p=0.01, f=0.001, q=0.5):
        """
        Initializes the model.

        n: number of rows/columns
        p: probability an empty cells becomes "forested" (occupied)
        f: probability of spontaneous fire (e.g., 1/fire ignition interval)
        q: initial forest density (probability cell is "forested" in initial state)

        """
        self.p = p
        self.f = f
        # initialize landscape with approx. q proportion of cells OCCUPIED
        self.state = np.random.choice([self.OCCUPIED, self.EMPTY], (n, n), p=[q, 1-q])


    def step(self):
        """
        Executes one time step, applying the CA rules to regenerate and burn forest.
        
        The code in this function was adapted from our Week 7 - Percolation
        notebook and Joseph's hints during the lab session.
        """
        # Get boolean arrays of empty and burning cells
        empty = self.state == self.EMPTY
        on_fire = self.state == self.FIRE



        # A cell with a tree burns if any of its neighbors is on fire
        hood_state = correlate2d(self.state, self.kernel, mode='same', boundary='wrap')  # calculate the neighbourhood state for each cell
        
        # TODO: Update at_risk to reflect level of risk for each cell (i.e., each cell represents a probability of catching fire in the next time step)
        #       - If a cell is empty, level of risk is 0.
        #       - If a cell is forested:
        #         - and none of its cardinal neighbours are on fire, level of risk is 0.
        #         - and at least one of its cardinal neighbours is on fire, level of risk depends on elevation.
        #           - if the forested cell is higher than the burning cell, level of risk is 1. (if elevation difference is too big, risk becomes 0)
        #           - if the forested cell is lower than the burning cell, level of risk is 0.5. (if elevation difference is too big, risk becomes 0)
        #           - if the forested cell is at the same elevation as the burning cell, level of risk is 0.7.
        at_risk = hood_state >= self.FIRE       # any cells surrounded by fire are at risk of burning
        
        forested = self.state == self.OCCUPIED  # get boolean array of forested cells
        
        ignite = at_risk & forested             # only at-risk cells that are forested can catch fire
        
        self.state[ignite] = self.FIRE



        # A cell with a tree spontaneously burns, with probability 𝑓, even if none of its neighbors is on fire
        forested = self.state == self.OCCUPIED  # update boolean array of forested cells
        self.state[forested] = np.random.choice(
            [self.FIRE, self.OCCUPIED],
            self.state[forested].shape,
            p=[self.f, 1-self.f]
        )

        # A cell with a burning tree becomes an empty cell in the next time step
        self.state[on_fire] = self.EMPTY

        # An empty cell becomes occupied with probability 𝑝
        self.state[empty] = np.random.choice(
            [self.OCCUPIED, self.EMPTY],
            self.state[empty].shape,
            p=[self.p, 1-self.p]
        )


    def num_occupied(self):
        """ Return the number of cells occupied by forest """
        n_occupied = np.sum(self.state == self.OCCUPIED)
        return n_occupied


    def pct_occupied(self):
        """ Return the proportion of cells occupied by forest. """
        pct_occupied = self.num_occupied() / self.state.size
        return pct_occupied


    def draw(self, axes=None, step=''):
        """ Draws the CA cells using the forest colour map so values are coloured intuitively. """
        axes = axes or plt.gca()
        title = f'Time: {step}, Occupied: {round(self.pct_occupied() * 100, 2)}%'
        plot_2D_array(self.state, axes=axes, title=title,
                    cmap=self.forest_colour_map, vmin=0, vmax=len(self.forest_colour_map.colors))

Create a miniature test forest to make sure the ForestFire model is working as expected.

In [5]:
# Code-a-little Test-a-little - use this space to test your class methods as you develop them.
small_fire = ForestFire(3, p=0.5, f=0.5)
small_fire.state = np.array([
    [0,0,0],
    [0,1,0],
    [1,1,1]
])

# Run 3 steps of the simulation to test the implementation of the rules
for i in range(3):
    small_fire.step()
    print(small_fire.state)

[[0 1 0]
 [1 5 0]
 [5 5 5]]
[[0 5 0]
 [5 0 1]
 [0 0 0]]
[[0 0 1]
 [0 0 5]
 [1 1 0]]


Create a simple animation (using Animate2D class provided) so we can visualize and verify the system dynamics

In [6]:
# Ex. 7.2 here
#  Suggestions: use a small grid (e.g., 20x20)  and
#               a short animation (100 frames) with interval=0.3, to slow it down so you can inspect its behaviour

test_fire = ForestFire(20)
fire_anim = Animation2D(test_fire, frames=25)
fire_anim.show()

## Build the Digital Elevation Model (DEM)

In [7]:
ones = np.ones((5,5))
print(ones)

slope = np.linspace(0, 4, 5)
print(slope)

dem1 = np.full((5,5), slope)
dem2 = np.full_like(ones, slope)
print(dem1)
print(dem2)

[[1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]
 [1. 1. 1. 1. 1.]]
[0. 1. 2. 3. 4.]
[[0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]]
[[0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]]


In [8]:
class Heightmap:
    """ Represents a heightmap for an inclined surface. """

    def __init__(self, n):
        """
        Initializes the heightmap.
        n: number of rows/columns
        """
        # initialize an n x n heightmap
        self.slope = np.linspace(0, n-1, n)
        self.state = np.full((n, n), self.slope)

test = Heightmap(5)
print(test.state)

[[0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]
 [0. 1. 2. 3. 4.]]


## Unpacking the Model

In [99]:
kernel = np.array([
    [0, 1, 0],
    [1, 0, 1],
    [0, 1, 0],
])

n = 5

state = np.zeros((n, n), dtype=int)
print(state)

forest = np.array([
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
    [1, 1, 1, 1, 1],
])

fire = np.array([
    [0, 0, 0, 0, 0],
    [0, 0, 0, 0, 0],
    [0, 0, 1, 0, 0],
    [0, 0, 0, 0, 0],
    [1, 0, 0, 0, 0],
])

elev = np.array([
    [0, 1, 2, 3, 4],
    [0, 1, 2, 3, 4],
    [0, 1, 2, 3, 4],
    [0, 1, 2, 3, 4],
    [0, 1, 2, 3, 4],
])

# Add the forest layer to state
state = state + forest
print(state)

# Add the fire layer to state
state[fire.astype(bool)] = 5
print(state)

neighbourhood = correlate2d(state, kernel, mode='same')
at_risk = neighbourhood >= 5
print(at_risk)

r = 0.75   # rate of fire spread across level ground
m = 0.25  # change in probabilty of fire spreading per unit slope

risk_factor = np.zeros(state.shape)
print(risk_factor)

ids = np.argwhere(fire)
for id in ids:
    d_elev = elev - elev[tuple(id)]
    risk = (((d_elev * m) + 1) * r)
    risk_factor[at_risk] += risk[at_risk]

print(risk_factor)

# burn_probs = r + ((elev-2)*r)
# print(burn_probs)

# probs = abs(at_risk*burn_probs)
# probs[probs >= 1] = 1
# print(probs)

# for row in range(5):
#     for col in range(5):
#         fire[row][col] = np.random.choice([1, 0], p=[probs[row][col], 1-probs[row][col]])
# print(fire)

[[0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]
 [0 0 0 0 0]]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 1 1 1]]
[[1 1 1 1 1]
 [1 1 1 1 1]
 [1 1 5 1 1]
 [1 1 1 1 1]
 [5 1 1 1 1]]
[[False False False False False]
 [False False  True False False]
 [False  True False  True False]
 [ True False  True False False]
 [False  True False False False]]
[[0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0.]]
[[0.    0.    0.    0.    0.   ]
 [0.    0.    1.875 0.    0.   ]
 [0.    1.5   0.    2.25  0.   ]
 [1.125 0.    1.875 0.    0.   ]
 [0.    1.5   0.    0.    0.   ]]
